# Loading a semantic map using knowrob

This notebook demonstrates how to load a semantic map into KnowRob

### Importing KnowRob Modules

We will first initialize KnowRob

In [1]:
import json
from knowrob import *
    
InitKnowRob()

[12:58:35.885] [info] [KnowRob] static initialization done.


### Setting Up Knowledge Base

Now we set up the knowledge base. See the second data-sources field, to see the path to an expected ontology. Add this ontology before running this cell.

In [2]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
			{"alias": "USD", "uri": "https://ease-crc.org/ont/USD.owl"},
            {"alias": "dfl", "uri": "http://www.ease-crc.org/ont/SOMA_DFL.owl"}

		]
	},
	"data-sources": [
		{"path": "owl/USD.owl", "format": "rdf-xml"},
		{"path": "owl/house_4.owl", "format": "rdf-xml"}
	],
	"data-backends": [
    {
      "type": "Redland",
      "name": "redland",
      "read-only": False
    }
	],  
    "reasoner": [
    {
      "type": "SOMADFLReasoner",
      "name": "DummyReasoner",
      "module": "reasoner/dfl_reasoner.py",
      "data-backend": "redland"
    }
    ]
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

[12:58:35.896] [info] Using backend `redland` with type `Redland`.
[12:58:35.898] [info] [redland] using storage of type "memory" with name "knowrob" and options "contexts='yes'".
[12:58:35.898] [info] Using queryable backend with id 'redland'.
[12:58:35.898] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/rdf-schema.xml' with version "Mon Apr 22 16:37:36 2024" and origin "rdf-schema".
[12:58:35.900] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/owl.rdf' with version "Mon Apr 22 16:37:36 2024" and origin "owl".
[12:58:35.902] [info] Loading ontology at '/home/sascha/.knowrob/owl/USD.owl' with version "Wed Aug 14 13:42:10 2024" and origin "USD".
[12:58:35.909] [info] Loading ontology at '/home/sascha/knowrob_ws/src/knowrob/owl/external/DUL.owl' with version "06-09-2024" and origin "DUL".
[12:58:35.928] [info] Loading ontology at '/home/sascha/.knowrob/owl/house_4.owl' with version "Tue Aug 20 12:42:54 2024" and origin "house_4".
[12:58:35.929] [

[12:58:36.119] [info] Using reasoner `DummyReasoner` with type `SOMADFLReasoner`.
[12:58:58.836] [info] Using goal-driven reasoner with id 'DummyReasoner'.


### Submitting a Query

Now we want to run a query on the loaded semantic map:

In [3]:
phi1 = QueryParser.parse("dfl:useMatch('dfl:serve.v.wn.consumption..concrete', ?x, 'dfl:breakfast_food.n.wn.food')")
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))


Received predicate  ''http://www.ease-crc.org/ont/SOMA_DFL.owl#useMatch''
UM 'dfl:serve.v.wn.consumption..concrete' 'dfl:serve.v.wn.consumption..concrete'
[12:59:04.226] [error] Reasoner 'DummyReasoner' produced an error in query evaluation. KeyError: "'dfl"
  File "/home/sascha/.knowrob/reasoner/dfl_reasoner.py", line 125, in evaluate
    task : Term = _iriOrVariable(self.reasoner, literal.arguments()[0])

  File "/home/sascha/.knowrob/reasoner/dfl_reasoner.py", line 8, in _iriOrVariable
    return IRIAtom(reasoner.expandName(str(x)))

  File "/home/sascha/Desktop/ease_lexical_resources/src/dfl/dlquery.py", line 87, in expandName
    retq = prefs[prefix] + suffix [∧ 'http://www.ease-crc.org/ont/SOMA_DFL.owl#useMatch'('dfl:serve.v.wn.consumption..concrete', ?x, 'dfl:breakfast_food.n.wn.food')]


In [4]:
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

while (not nextResult1.indicatesEndOfEvaluation()):
    if isinstance(nextResult1, AnswerYes):
        for substitution in nextResult1.substitution():
            variable = substitution[1]
            term = substitution[2]
            print(str(variable) + " : " + str(term) + ";\n")
    elif isinstance(nextResult1, AnswerNo):
        print("False")
    elif isinstance(nextResult1, AnswerDontKnow):
        print("Don't Know")
    nextResult1 = resultQueue.pop_front()

False


CQ1 Which objects do I need for breakfast/
CQ2 Which objects contain something to drink?
CQ3 Where do we expect an item to be?
CQ4 What can I grasp on an object to open it?
CQ5 Where should I put the utensils for breakfast?

In [5]:
query = "dfl:isInstanceOf(?x, 'http://www.ease-crc.org/ont/SOMA_DFL.owl#furniture.n.wn.artifact'), \
    dfl:isSubclassOf(?z, 'http://www.ease-crc.org/ont/SOMA_DFL.owl#furniture.n.wn.artifact')"
phi1 = QueryParser.parse(query)
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

while (not nextResult1.indicatesEndOfEvaluation()):
    if isinstance(nextResult1, AnswerYes):
        for substitution in nextResult1.substitution():
            variable = substitution[1]
            term = substitution[2]
            print(str(variable) + " : " + str(term) + ";\n")
    elif isinstance(nextResult1, AnswerNo):
        print("False")
    elif isinstance(nextResult1, AnswerDontKnow):
        print("Don't Know")
    nextResult1 = resultQueue.pop_front()

Received predicate  dfl:isSubclassOf
	 s ?z
	 o dfl:furniture.n.wn.artifact
Bounding (True, False)
Concepts ['http://www.ease-crc.org/ont/SOMA_DFL.owl#airing_cupboard.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#altar.n.wn.artifact..communion', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#armchair.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#armoire.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#baby_bed.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#banquette.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#bar.n.wn.artifact..counter', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#barber_chair.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#bassinet.n.wn.artifact..baby_bed', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#bath_chair.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#bed.n.wn.artifact..furniture', 'http://www.ease-crc.org/ont/SOMA_DFL.owl#bedroom_furniture.n.wn.artifact', 'http://www.ease-crc.org/ont/SOMA_DFL

Instances {'USD:Sofa_7_1'}
Received predicate  dfl:isInstanceOf
	 s ?x
	 o dfl:furniture.n.wn.artifact
Bounding (True, False)
False
[12:59:05.716] [warning] Checking dfl:isInstanceOf(?x, dfl:furniture.n.wn.artifact)
Concepts ['https://ease-crc.org/ont/USD.owl#Sofa_7_1']
Instances {'https://ease-crc.org/ont/USD.owl#Sofa_7_1'}
False
